# Capstone Project Mata Kuliah Jaringan Syaraf Tiruan

Dosen Pengampu: Mochammad Anshori S.Kom.,M.Kom.

#1. Import Library


In [1]:
#Install Library yang dibutuhkan

!pip install torch_geometric               # Menyediakan tools dan layers yang diperlukan untuk mengimplementasikan GNN
!pip install optuna                        # Hyperparameter optimization

#2. Data Preparation

In [2]:
import pandas as pd
import torch
from torch_geometric.data import Data

# Memuat file edges dan nodes/target
edges_df = pd.read_csv('/content/drive/MyDrive/Jaringan Syaraf Tiruan/musae_git_edges.csv')  # Memuat file edges
nodes_df = pd.read_csv('/content/drive/MyDrive/Jaringan Syaraf Tiruan/musae_git_target.csv')  # Memuat file nodes dengan target

# Membuat edge_index dari kolom 'id_1' dan 'id_2'
edge_index = torch.tensor(edges_df.values.T, dtype=torch.long)

# Membuat fitur node (asumsikan hanya menggunakan id sebagai fitur numerik sederhana)
x = torch.tensor(nodes_df['id'].values.reshape(-1, 1), dtype=torch.float)

# Membuat target label (ml_target)
y = torch.tensor(nodes_df['ml_target'].values, dtype=torch.long)

# Membuat objek Data untuk GNN
data = Data(x=x, edge_index=edge_index, y=y)


In [3]:
# Menampilkan jumlah node
num_nodes = data.num_nodes
print(f"Number of nodes: {num_nodes}")

Number of nodes: 37700


In [4]:
# Menampilkan jumlah edge
num_edges = data.edge_index.shape[1]
print(f"Number of edges: {num_edges}")

Number of edges: 289003


#3. Memilih Arsitektur GNN dan Desain Model
Pada kasus ini menggunakan Arsitektur *Graph Convolutional Network* (GCN)

In [5]:
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gcn1 = GCNConv(input_dim, hidden_dim)  # Layer konvolusi pertama
        self.gcn2 = GCNConv(hidden_dim, output_dim)  # Layer konvolusi kedua
        self.fc = nn.Linear(output_dim, 2)  # Output layer (2 kelas: Web Developer / ML Developer)

    def forward(self, x, edge_index):
        x = self.gcn1(x, edge_index).relu()  # Aktivasi ReLU pada layer pertama
        x = self.gcn2(x, edge_index).relu()  # Aktivasi ReLU pada layer kedua
        x = self.fc(x)  # Output layer
        return x


#4. Melatih dan Mengevaluasi Performa Model

In [6]:
import torch.optim as optim
import torch.nn.functional as F

# Inisialisasi model, optimizer, dan loss function
model = GCN(input_dim=data.x.shape[1], hidden_dim=16, output_dim=16)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Fungsi pelatihan
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)  # Forward pass
    loss = criterion(out, data.y)  # Menghitung loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update bobot
    return loss.item()

# Fungsi evaluasi
def evaluate(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)  # Prediksi
        pred = out.argmax(dim=1)  # Kelas dengan probabilitas tertinggi
        correct = (pred == data.y).sum().item()  # Hitung prediksi yang benar
        accuracy = correct / len(data.y)
    return accuracy

# Melatih model
for epoch in range(1, 201):
    loss = train(model, data, optimizer, criterion)
    if epoch % 20 == 0:
        acc = evaluate(model, data)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Accuracy: {acc:.4f}')

# Evaluasi akhir setelah pelatihan selesai
accuracy = evaluate(model, data)
print(f'\n\nAkurasi Model: {accuracy:.4f}')

Epoch: 020, Loss: 156.3845, Accuracy: 0.7417
Epoch: 040, Loss: 14.5254, Accuracy: 0.7415
Epoch: 060, Loss: 53.9478, Accuracy: 0.2583
Epoch: 080, Loss: 24.4573, Accuracy: 0.7415
Epoch: 100, Loss: 10.7834, Accuracy: 0.2583
Epoch: 120, Loss: 42.3189, Accuracy: 0.7415
Epoch: 140, Loss: 0.7545, Accuracy: 0.7411
Epoch: 160, Loss: 16.5604, Accuracy: 0.7411
Epoch: 180, Loss: 0.6802, Accuracy: 0.2583
Epoch: 200, Loss: 11.6381, Accuracy: 0.7409


Akurasi Model: 0.7409
